In [1]:
%cd /content/drive/MyDrive/Colab Notebooks/Code/NLP_robotics/LTL_dataset/github
!pip install transformers
!pip install SentencePiece

/content/drive/MyDrive/Colab Notebooks/Code/NLP_robotics/LTL_dataset/github
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 33.9 MB/s 
     |████████████████████████████████| 182 kB 65.5 MB/s 
     |████████████████████████████████| 7.6 MB 49.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 34.6 MB/s 


In [2]:
import os
import json
import time
from argparse import ArgumentParser
import tqdm
import torch
import torch.nn as nn

from torch.utils.data import DataLoader

from transformers import T5Tokenizer, AdamW, get_linear_schedule_with_warmup
from seq2seq import LTL2Eng
from data import LTLDataset, generate_vocabs, LTL2EngT5Dataset, UnlabeledLTLDataset
from config import Config
from util import collect_ltl_vocabs

config = Config.from_json_file('eng2ltl_t5_load_data.json')
input_dir = os.path.join(config.log_path, 'eng2ltl_para_gen_5_11_28_word_infix')

torch.cuda.set_device(0)
model_name = config.bert_model_name
tokenizer = T5Tokenizer.from_pretrained(model_name,cache_dir=config.bert_cache_dir)

ltl_vocabs = collect_ltl_vocabs([config.train_file,config.dev_file,config.test_file])
model = LTL2Eng(config,ltl_vocabs,tokenizer)
model.load_state_dict(torch.load(input_dir+'/model_state.pt'))
model.cuda(device=0)
model.eval()

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

"\n# test\nprogress = tqdm.tqdm(total=test_batch_num, ncols=75,\n                    desc='Test {}'.format(1))\n# calculate the one on one match accuracy\ncount_3 = 0\ntotal = 0\ntest_predictions = []\nfor batch in DataLoader(test_set, batch_size=config.eval_batch_size,\n                        shuffle=False, collate_fn=test_set.collate_fn):\n    progress.update(1)\n    with torch.no_grad():\n        outputs = model.predict_eng2ltl(batch, max_length=config.max_generate_length)\n        pred_ltls = tokenizer.batch_decode(outputs['output_idxs'], skip_special_tokens=True)\n\n        for pred_ltl, gold_sent, gold_ltl in zip(pred_ltls, outputs['tokens'], outputs['ltl']):\n            print('-----------------------------------')\n            print('sent: ', ' '.join(gold_sent))\n            print('gold: ',' '.join(gold_ltl))\n            print('pred: ',pred_ltl)\n\n            item_pred = pred_ltl.strip().split(' ')\n            if item_pred == gold_ltl:\n                count_3 += 1\n      

In [3]:
test_sentence = ['If ( prop_4 ) happens and implies ( prop_2 ) and this scenario continues to hold until at some point during the 243 to 582 time units ( prop_3 ) is detected , then the scenario is equivalent to ( prop_1 ) .',
                 'If it is not the case that ( prop_3 ) is detected for each time instant in the coming 164 to 612 time units , or else ( prop_1 ) happens , then ( prop_2 ) .',
                 'If at some point ( prop_3 ) and ( prop_2 ) , and is equivalent to ( prop_4 ) , and this scenario will hold until at some other point ( prop_1 ) is detected .',
                 
                 
                 ]

In [4]:
import os
home_path_output = '/content/drive/MyDrive/Colab Notebooks/Code/NLP_robotics/LTL_dataset/github/application_test/'
if not os.path.exists(home_path_output):
  os.mkdir(home_path_output)

dataset_total = [];
with open(home_path_output+"test1_word_infix.jsonl", "w") as outfile:
    for i in range(len(test_sentence)):
      dataset_item = {};
      dataset_item['id'] = i
      dataset_item['logic_ltl'] = ''
      dataset_item['logic_sentence'] = test_sentence[i].split(' ')
      outfile.write(json.dumps(dataset_item)+'\n')
outfile.close()

test_set = LTL2EngT5Dataset(home_path_output+"test1_word_infix.jsonl",tokenizer,config,model.ltl2eng.config,ltl_vocabs=ltl_vocabs)
data_batch =  DataLoader(test_set, batch_size=1,
                        shuffle=False, collate_fn=test_set.collate_fn)

3it [00:00, 228.95it/s]

Loaded 3 instances from /content/drive/MyDrive/Colab Notebooks/Code/NLP_robotics/LTL_dataset/github/application_test/test1_word_infix.jsonl


In [5]:
for i, batch in enumerate(data_batch):
  if i <len(test_sentence):
    outputs = model.predict_eng2ltl(batch, max_length=config.max_generate_length)
    pred_ltls = tokenizer.batch_decode(outputs['output_idxs'], skip_special_tokens=True)
    print(test_sentence[i])
    print(pred_ltls)
    print('\n')
  else:
    break

If ( prop_4 ) happens and implies ( prop_2 ) and this scenario continues to hold until at some point during the 243 to 582 time units ( prop_3 ) is detected , then the scenario is equivalent to ( prop_1 ) .
['( ( ( prop_4 imply prop_2 ) until [243,582] prop_3 ) equal prop_1 )']


If it is not the case that ( prop_3 ) is detected for each time instant in the coming 164 to 612 time units , or else ( prop_1 ) happens , then ( prop_2 ) .
['( ( negation prop_3 imply prop_1 ) or prop_2 )']


If at some point ( prop_3 ) and ( prop_2 ) , and is equivalent to ( prop_4 ) , and this scenario will hold until at some other point ( prop_1 ) is detected .
['( ( ( prop_3 and prop_2 ) equal prop_4 ) until prop_1 )']


